In [17]:
#!/usr/bin/env python

import numpy as np
import pandas as pd


nInst = 0
nt = 0
commRate = 0.0010
dlrPosLimit = 10000


def loadPrices(fn):
    global nt, nInst
    df = pd.read_csv(fn, sep='\s+', header=None, index_col=None)
    (nt, nInst) = df.shape
    return (df.values).T


pricesFile = "./prices.txt"
prcAll = loadPrices(pricesFile)
print("Loaded %d instruments for %d days" % (nInst, nt))


Loaded 50 instruments for 750 days


In [18]:
nInst = 50
currentPos = np.zeros(nInst)

def getMyPosition(prcSoFar):
    global currentPos
    (nins, nt) = prcSoFar.shape
    
    if nt < 30:  # We need at least 30 days of data
        return np.zeros(nins)
    
    # Calculate z-score of prices
    lookback = 20
    means = np.mean(prcSoFar[:, -lookback:], axis=1)
    stds = np.std(prcSoFar[:, -lookback:], axis=1)
    z_scores = (prcSoFar[:, -1] - means) / stds
    
    # Generate signals based on z-scores
    signals = -z_scores  # Negative z-score means we expect price to increase
    
    # Normalize signals
    signals = signals / np.sum(np.abs(signals))
    
    # Calculate target positions
    target_position = np.array([int(x) for x in dlrPosLimit * 0.5 * signals / prcSoFar[:, -1]])
    
    # Implement a conservative position change
    position_change = 0.2 * (target_position - currentPos)
    
    # Update current position
    currentPos = np.array([int(x) for x in currentPos + position_change])
    
    # Ensure we're within position limits
    current_prices = prcSoFar[:, -1]
    posLimits = np.array([int(x) for x in dlrPosLimit / current_prices])
    currentPos = np.clip(currentPos, -posLimits, posLimits)
    
    return currentPos

# The rest of the code (calcPL function and the main execution) remains the same


In [19]:
def calcPL(prcHist):
    cash = 0
    curPos = np.zeros(nInst)
    totDVolume = 0
    totDVolumeSignal = 0
    totDVolumeRandom = 0
    value = 0
    todayPLL = []
    (_, nt) = prcHist.shape
    for t in range(500, 751):
        prcHistSoFar = prcHist[:, :t]
        newPosOrig = getMyPosition(prcHistSoFar)
        curPrices = prcHistSoFar[:, -1]
        posLimits = np.array([int(x) for x in dlrPosLimit / curPrices])
        newPos = np.clip(newPosOrig, -posLimits, posLimits)
        deltaPos = newPos - curPos
        dvolumes = curPrices * np.abs(deltaPos)
        dvolume = np.sum(dvolumes)
        totDVolume += dvolume
        comm = dvolume * commRate
        cash -= curPrices.dot(deltaPos) + comm
        curPos = np.array(newPos)
        posValue = curPos.dot(curPrices)
        todayPL = cash + posValue - value
        todayPLL.append(todayPL)
        value = cash + posValue
        ret = 0.0
        print(newPos)
        if (totDVolume > 0):
            ret = value / totDVolume
        print("Day %d value: %.2lf todayPL: $%.2lf $-traded: %.0lf return: %.5lf" %
              (t, value, todayPL, totDVolume, ret))
    pll = np.array(todayPLL)
    (plmu, plstd) = (np.mean(pll), np.std(pll))
    annSharpe = 0.0
    if (plstd > 0):
        annSharpe = np.sqrt(250) * plmu / plstd
    return (plmu, ret, plstd, annSharpe, totDVolume)


(meanpl, ret, plstd, sharpe, dvol) = calcPL(prcAll)
score = meanpl - 0.1*plstd



[ 0  0  0  0  0  1 -1  0  0  0  0 -1  0  0 -2  0  0  0  2  0  0  0  0  1
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0
  0  0]
Day 500 value: -0.15 todayPL: $-0.15 $-traded: 150 return: -0.00100
[ 0  0  0  0  0  3 -1  0  0  0  0 -1  1  0 -4  0  0  0  3  0  0  0  0  1
  0  0  0  0  0  0  1  0  0  0  0  0  0  0  1  0  0  0  3  0  0  0  0  0
  0  0]
Day 501 value: -0.77 todayPL: $-0.62 $-traded: 278 return: -0.00276
[ 1  0  0  0  0  5 -1  0  0  0  0 -1  2  0 -5  0  0  0  4  0  0  0  0  1
  0  0  0  0  0  0  1  0  0  0 -1  0  0  0  2  0  0  0  3  0  0  0  0  0
  0  0]
Day 502 value: -1.15 todayPL: $-0.38 $-traded: 406 return: -0.00282
[ 0  0  0  0  0  7 -2  0  0  0  0 -1  3  0 -6  0  0  0  4  0  0  0  0  2
  0  0  0  0  0  0  1  0  0  0 -1  0  0  0  2  0  0  0  3  0  0  0  0  0
  0  0]
Day 503 value: -2.38 todayPL: $-1.24 $-traded: 522 return: -0.00457
[ 0  0  0  0  0  8 -3  0  0  0  0 -1  3  0 -5  0  0  0  4  0  0  0  0  2
  0  0  0  0  0  0  3  0  0  0 -2  0  0

In [20]:
print("=====")
print("mean(PL): %.1lf" % meanpl)
print("return: %.5lf" % ret)
print("StdDev(PL): %.2lf" % plstd)
print("annSharpe(PL): %.2lf " % sharpe)
print("totDvolume: %.0lf " % dvol)
print("Score: %.2lf" % score)

=====
mean(PL): -0.4
return: -0.00246
StdDev(PL): 1.57
annSharpe(PL): -4.46 
totDvolume: 44988 
Score: -0.60
